In [ ]:
import yaml
import cv2
import numpy as np
import PIL.Image
import time
from pynq.lib.video import *
from pynq.lib import Pmod_PWM
from pynq.lib.arduino import Arduino_IO
from pynq.overlays.base import BaseOverlay

In [ ]:
## Formate output frame for HDMI output ##

def format_frame(hdmi_out, frame):
    
    # Read input frame size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    #print(frame.shape)
    
    # Creat new video formate frame
    outframe = hdmi_out.newframe()
    
    # Fill new frame with zero
    zero_img = np.zeros((480, 640), dtype=np.uint8)
    outframe[0:480,0:640] = zero_img[0:480,0:640]

    # Transfer array to the video frame
    # For Gray imge
    outframe[0:img_h,0:img_w] = frame[0:img_h,0:img_w]
    # For RBG image
    #outframe[0:480,0:640,:] = edges[0:480,0:640,:]
    
    return outframe

In [ ]:
## Edge Detection ##

def edge_detection(frame):
    
    # Read input img size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    
    # Canny detect edges
    edges = cv2.Canny(frame, 39, 130)

    return edges

In [ ]:
## Initialize HDMI I/O ##

# Load the overlay
base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure HDMI input to gray scale ( (0.3 * R) + (0.59 * G) + (0.11 * B) )
hdmi_in.configure(PIXEL_GRAY)
hdmi_in.start()

# Configure Output resolution (w, h, bit per pixek)
hdmi_out_mode = VideoMode(640,480,8)
hdmi_out.configure(hdmi_out_mode, PIXEL_GRAY)
hdmi_out.start()

In [ ]:
## Read camera calibration parameter ##

fname = "calibration_parameter.yaml"
with open(fname) as file:
    data = yaml.load(file,Loader=yaml.Loader)
    
mtx = np.array([ [ data[0] , 0, data[1] ] , [ 0, data[2], data[3] ] , [0, 0, 1] ])
dist = np.array([ [ data[4], data[5], data[6], data[7], data[8] ] ])

In [ ]:
## Main ##

while(True):
    # Read frame from HDMI input
    frame = hdmi_in.readframe()
    
    # Resize frame (half of the 720p input)
    img = cv2.resize(frame, (640,360))
    
    # Undistort input frame
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    # Crop the frame baseo on undistortion
    x,y,w,h = roi
    #dst = dst[y:y+h, x:x+w]
    
    # Crop the frame to align center of the frame
    dst = dst[y:y+h, x+120:x+w-150]

    # Dection line
    overlay = edge_detection(dst)
    
    outframe = format_frame(hdmi_out, overlay)
    
    hdmi_out.writeframe(outframe)   

In [ ]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out